In [1]:
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from itertools import combinations
import seaborn as sns
from sklearn import metrics
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.feature_selection import RFE
from bayes_opt import BayesianOptimization

In [2]:
data=pd.read_excel(r"C:\Users\HP\Desktop\Data.xlsx",sheet_name='16+3',index_col=0,header=0)
data1=data.iloc[0:16, :]

minmax_scaler=preprocessing.MinMaxScaler()  
data2=minmax_scaler.fit_transform(data1)
data3=pd.DataFrame(data2,columns=['O3_input','lg(O3)','H2O2_input','lg(H2O2)','H2O2/O3','lg(H2O2/O3)','pH','volumn',
                                  'TOC','UV254','Φn1','Φn2','Φn3','Φn4','Φn5','FRI','FRI20','OU20','OU60','OU120',
                                 'O3(1)20','O3(1)60','O3(1)120','O3(g)20','O3(g)60','O3(g)120','H2O2_20','H2O2_60',
                                  'H2O2_120','FMax1','FMax2','FMax3','FMax4','cost'])
data4=data3[['lg(O3)','lg(H2O2)','pH','TOC','UV254','Φn1','Φn2','Φn3','Φn4','Φn5','FRI','OU20','OU60','OU120',
                                 'O3(1)20','O3(1)60','O3(1)120','O3(g)20','O3(g)60','O3(g)120','H2O2_20','H2O2_60',
                                  'H2O2_120','FMax1','FMax2','FMax3','FMax4']]

,O3_input,lg(O3),H2O2_input,lg(H2O2),H2O2/O3,lg(H2O2/O3),pH,volumn,TOC,UV254,...,O3(g)60,O3(g)120,H2O2_20,H2O2_60,H2O2_120,FMax1,FMax2,FMax3,FMax4,cost
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.174272,0.276007,...,0.003091,0.003882,0.022302,0.041805,0.000000,0.291770,0.338220,0.197112,0.346430,0.000000
1,0.000000,0.000000,0.019382,0.469188,1.000000,1.000000,0.333333,0.0,0.049316,0.130054,...,0.002041,0.001648,0.127310,1.000000,0.135055,0.261372,0.179775,0.334419,0.153176,0.050139
2,0.000000,0.000000,0.002931,0.234969,0.151232,0.500800,0.666667,0.0,0.085820,0.000000,...,0.003283,0.003787,0.026079,0.051821,0.000167,0.000000,0.000000,0.000000,0.000000,0.013243
3,0.000000,0.000000,0.019382,0.469188,1.000000,1.000000,1.000000,0.0,0.000000,0.495418,...,0.000000,0.000000,0.040433,0.207068,0.022114,0.570154,0.465007,0.624370,0.519317,0.124692
4,0.183673,0.588592,0.005673,0.312432,0.000000,0.000000,0.000000,0.0,0.094595,0.519975,...,0.017796,0.022048,0.016258,0.045143,0.006895,0.793536,0.648313,0.799742,0.747103,0.371966
5,0.183673,0.588592,0.005673,0.312432,0.000000,0.000000,0.333333,0.0,0.587224,0.564706,...,0.165586,0.181925,0.005870,0.518828,0.074685,0.996455,1.000000,1.000000,0.983669,0.102126
6,0.183673,0.588592,0.034985,0.547401,0.151232,0.500800,0.333333,0.0,0.696736,0.607049,...,0.146710,0.162042,0.007638,0.799496,0.118716,0.974190,0.950282,0.987169,0.969692,0.095574
7,0.183673,0.588592,0.116579,0.709029,0.572205,0.845285,0.333333,0.0,0.803263,1.000000,...,0.083098,0.085491,0.111194,0.150287,0.009459,0.994320,0.975092,0.980770,0.979919,0.109757
8,0.183673,0.588592,0.199496,0.781620,1.000000,1.000000,0.333333,0.0,0.548614,0.848700,...,0.049286,0.052538,0.542431,0.647771,0.031726,0.954510,0.925464,0.955066,0.900332,0.197980
9,0.183673,0.588592,0.199496,0.781620,1.000000,1.000000,0.666667,0.0,0.559319,0.835121,...,0.038794,0.041300,0.127864,0.477572,0.017636,0.981925,0.984824,0.981197,0.979841,0.192236


In [4]:
def MODEL(X, y):
    def black_box_function(C, gamma):
        model = SVR(kernel='rbf', C=C, gamma=gamma)
        loo = LeaveOneOut()
        y_real = []
        y_predicted = []

        # Applying LOO-CV
        for train_index, test_index in loo.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            model.fit(X_train, y_train.ravel())
            y_pred = model.predict(X_test)
            y_real.extend(y_test)
            y_predicted.extend(y_pred)

        res = r2_score(y_real, y_predicted)
        return res

    pbounds = {'C': (0.1, 10),
               'gamma': (0.0001, 1)}
    optimizer = BayesianOptimization(
        f=black_box_function,
        pbounds=pbounds,
        verbose=2,
        random_state=1)
    optimizer.maximize(
        init_points=20,
        n_iter=30)
    maxx = optimizer.max
    maxx2 = maxx['target']
    return maxx2


data7 = data4.values


column_names = data4.columns

X_original = data7[:, [i for i in range(data7.shape[1]) if i!= 3]]
y_original = data7[:, 3]


initial_features = X_original[:, :3]

remaining_indices = range(3, X_original.shape[1])
all_combinations = list(combinations(remaining_indices, 2))

all_results = []

for comb in all_combinations:
    new_feature_subset = np.hstack((initial_features,
                                    X_original[:, comb[0]].reshape(-1, 1),
                                    X_original[:, comb[1]].reshape(-1, 1)))

    maxx2 = MODEL(new_feature_subset, y_original)

    all_results.append((maxx2, new_feature_subset, [0, 1, 2] + list(comb)))

all_results.sort(key=lambda x: x[0], reverse=True)

# Outputs the top 5 highest maxx2 values, corresponding feature subsets and feature indexes.
for i in range(5):
    print(f"top {i + 1}maxx2 value:", all_results[i][0])

    data7_indexes = [index if index < 3 else index + 1 for index in all_results[i][2]]

    print(f"The corresponding feature subset:", all_results[i][1])
    print(f"The corresponding index in data7:", data7_indexes)
    print(f"The corresponding variable name:", [column_names[index] for index in data7_indexes])

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
| 1         | 0.3935    | 4.229     | 0.7204    |
| 2         | 0.1807    | 0.1011    | 0.3024    |
| 3         | 0.3279    | 1.553     | 0.09243   |
| 4         | 0.435     | 1.944     | 0.3456    |
| 5         | 0.3768    | 4.028     | 0.5389    |
| 6         | 0.3909    | 4.25      | 0.6853    |
| 7         | 0.3921    | 2.124     | 0.8781    |
| 8         | 0.3148    | 0.3711    | 0.6705    |
| 9         | 0.3726    | 4.231     | 0.5587    |
| 10        | 0.3349    | 1.49      | 0.1982    |
| 11        | 0.395     | 8.027     | 0.9683    |
| 12        | 0.3808    | 3.203     | 0.6924    |
| 13        | 0.3937    | 8.776     | 0.8946    |
| 14        | 0.281     | 0.9419    | 0.03915   |
| 15        | 0.4003    | 1.781     | 0.8782    |
| 16        | 0.3817    | 1.074     | 0.4212    |
| 17        | 0.3897    | 9.583     | 0.5332    |
| 18        | 0.3895    | 6.95      | 0.3156    |


| 40        | 0.601     | 5.038     | 0.7216    |
| 41        | 0.5696    | 5.693     | 0.118     |
| 42        | -0.0366   | 4.86      | 0.0001    |
| 43        | 0.5726    | 5.824     | 0.1451    |
| 44        | -0.04192  | 2.59      | 0.0001    |
| 45        | 0.595     | 6.374     | 1.0       |
| 46        | 0.595     | 4.731     | 1.0       |
| 47        | -0.03272  | 6.523     | 0.0001    |
| 48        | 0.603     | 5.947     | 0.5966    |
| 49        | -0.02862  | 8.287     | 0.0001    |
| 50        | 0.595     | 3.641     | 1.0       |
|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
| 1         | 0.4304    | 4.229     | 0.7204    |
| 2         | 0.2957    | 0.1011    | 0.3024    |
| 3         | 0.4579    | 1.553     | 0.09243   |
| 4         | 0.5182    | 1.944     | 0.3456    |
| 5         | 0.4264    | 4.028     | 0.5389    |
| 6         | 0.427     | 4.25      | 0.6853    |
| 7         | 0.466     | 2.124     | 0.8781    |


| 29        | -0.04219  | 2.622     | 0.0001    |
| 30        | 0.5295    | 1.138     | 0.2272    |
| 31        | 0.5019    | 3.22      | 0.3518    |
| 32        | 0.4515    | 8.363     | 0.2658    |
| 33        | 0.5019    | 6.172     | 1.0       |
| 34        | 0.5522    | 1.338     | 0.3741    |
| 35        | 0.4602    | 5.658     | 0.4244    |
| 36        | -0.02599  | 10.0      | 0.0001    |
| 37        | 0.5019    | 9.417     | 1.0       |
| 38        | 0.5019    | 5.104     | 1.0       |
| 39        | -0.0371   | 4.923     | 0.0001    |
| 40        | 0.4907    | 5.038     | 0.7216    |
| 41        | -0.03413  | 6.275     | 0.0001    |
| 42        | 0.4695    | 7.566     | 0.4504    |
| 43        | 0.5177    | 5.824     | 0.1451    |
| 44        | 0.4693    | 9.037     | 0.3689    |
| 45        | 0.5019    | 7.426     | 1.0       |
| 46        | 0.5019    | 10.0      | 1.0       |
| 47        | 0.4929    | 2.683     | 1.0       |
| 48        | -0.03994  | 3.639     | 0.0001    |


| 20        | 0.5215    | 0.2811    | 0.7502    |
| 21        | 0.5929    | 1.309     | 0.3892    |
| 22        | 0.5785    | 1.574     | 0.4641    |
| 23        | 0.5575    | 3.943     | 0.4893    |
| 24        | 0.575     | 1.141     | 0.6682    |
| 25        | 0.5959    | 0.7254    | 0.6769    |
| 26        | 0.5737    | 0.6148    | 0.9545    |
| 27        | 0.5739    | 2.321     | 0.4135    |
| 28        | 0.558     | 2.674     | 0.6137    |
| 29        | 0.5107    | 2.639     | 0.1726    |
| 30        | 0.5181    | 1.138     | 0.2272    |
| 31        | 0.56      | 3.22      | 0.3518    |
| 32        | 0.5459    | 0.9613    | 1.0       |
| 33        | 0.474     | 3.603     | 0.1442    |
| 34        | 0.52      | 7.721     | 0.5116    |
| 35        | 0.5258    | 2.849     | 0.8594    |
| 36        | 0.4623    | 4.293     | 0.1395    |
| 37        | 0.5639    | 2.06      | 0.5769    |
| 38        | 0.5201    | 3.65      | 0.7818    |
| 39        | -0.03924  | 6.618     | 0.0001    |


| 13        | 0.4831    | 8.776     | 0.8946    |
| 14        | 0.2832    | 0.9419    | 0.03915   |
| 15        | 0.4895    | 1.781     | 0.8782    |
| 16        | 0.5284    | 1.074     | 0.4212    |
| 17        | 0.4288    | 9.583     | 0.5332    |
| 18        | 0.3801    | 6.95      | 0.3156    |
| 19        | 0.4782    | 6.896     | 0.8346    |
| 20        | 0.4615    | 0.2811    | 0.7502    |
| 21        | 0.5269    | 1.412     | 0.2989    |
| 22        | 0.5388    | 1.583     | 0.3452    |
| 23        | 0.4222    | 3.943     | 0.4893    |
| 24        | 0.5169    | 1.796     | 0.4912    |
| 25        | 0.4329    | 6.312     | 0.5852    |
| 26        | 0.4912    | 2.043     | 0.5317    |
| 27        | 0.532     | 1.207     | 0.5848    |
| 28        | 0.5332    | 0.983     | 0.6289    |
| 29        | 0.5231    | 0.6907    | 0.6611    |
| 30        | 0.5322    | 1.138     | 0.2272    |
| 31        | 0.4908    | 3.22      | 0.3518    |
| 32        | 0.5115    | 3.482     | 0.2753    |


...

| 14        | 0.3396    | 0.9419    | 0.03915   |
| 15        | 0.3058    | 1.781     | 0.8782    |
| 16        | 0.4857    | 1.074     | 0.4212    |
| 17        | 0.2515    | 9.583     | 0.5332    |
| 18        | 0.4406    | 6.95      | 0.3156    |
| 19        | 0.1548    | 6.896     | 0.8346    |
| 20        | 0.3377    | 0.2811    | 0.7502    |
| 21        | 0.4717    | 1.454     | 0.4662    |
| 22        | 0.5176    | 1.279     | 0.2577    |
| 23        | 0.5164    | 2.288     | 0.209     |
| 24        | -0.04347  | 2.602     | 0.0001    |
| 25        | 0.4697    | 2.228     | 0.4326    |
| 26        | 0.4661    | 2.07      | 0.1028    |
| 27        | 0.408     | 0.903     | 0.8011    |
| 28        | -0.0355   | 7.186     | 0.0001    |
| 29        | 0.4654    | 6.588     | 0.2847    |
| 30        | 0.506     | 1.138     | 0.2272    |
| 31        | 0.4816    | 3.22      | 0.3518    |
| 32        | 0.5041    | 3.515     | 0.1557    |
| 33        | -0.04123  | 3.887     | 0.0001    |


| 15        | 0.2814    | 1.781     | 0.8782    |
| 16        | 0.4607    | 1.074     | 0.4212    |
| 17        | 0.2163    | 9.583     | 0.5332    |
| 18        | 0.4075    | 6.95      | 0.3156    |
| 19        | 0.1181    | 6.896     | 0.8346    |
| 20        | 0.3225    | 0.2811    | 0.7502    |
| 21        | 0.459     | 1.442     | 0.5124    |
| 22        | -0.03669  | 6.632     | 0.0001    |
| 23        | 0.3986    | 0.9096    | 0.8134    |
| 24        | 0.443     | 7.328     | 0.2349    |
| 25        | 0.4585    | 2.343     | 0.1542    |
| 26        | -0.04323  | 2.795     | 0.0001    |
| 27        | -0.04444  | 2.088     | 0.0001    |
| 28        | 0.4662    | 1.697     | 0.4124    |
| 29        | 0.4658    | 2.295     | 0.4082    |
| 30        | 0.4491    | 1.138     | 0.2272    |
| 31        | 0.4664    | 3.22      | 0.3518    |
| 32        | 0.4634    | 3.496     | 0.259     |
| 33        | -0.0416   | 3.752     | 0.0001    |
| 34        | 0.4625    | 2.342     | 0.1602    |


| 16        | 0.613     | 1.074     | 0.4212    |
| 17        | 0.621     | 9.583     | 0.5332    |
| 18        | 0.5293    | 6.95      | 0.3156    |
| 19        | 0.6446    | 6.896     | 0.8346    |
| 20        | 0.5336    | 0.2811    | 0.7502    |
| 21        | 0.5698    | 1.393     | 0.5911    |
| 22        | 0.6371    | 8.418     | 0.929     |
| 23        | 0.6221    | 8.577     | 0.5819    |
| 24        | 0.6407    | 9.102     | 0.6275    |
| 25        | 0.6291    | 9.26      | 0.9884    |
| 26        | 0.6275    | 7.258     | 1.0       |
| 27        | 0.6275    | 6.54      | 1.0       |
| 28        | 0.5963    | 8.033     | 0.5205    |
| 29        | 0.5263    | 9.177     | 0.1671    |
| 30        | 0.5773    | 1.138     | 0.2272    |
| 31        | 0.5447    | 3.22      | 0.3518    |
| 32        | 0.6332    | 9.746     | 0.9563    |
| 33        | 0.6523    | 10.0      | 0.6302    |
| 34        | 0.6446    | 6.901     | 0.8383    |
| 35        | 0.5156    | 10.0      | 0.205     |


| 16        | 0.5882    | 1.074     | 0.4212    |
| 17        | 0.5728    | 9.583     | 0.5332    |
| 18        | 0.5249    | 6.95      | 0.3156    |
| 19        | 0.6087    | 6.896     | 0.8346    |
| 20        | 0.5214    | 0.2811    | 0.7502    |
| 21        | 0.5914    | 1.17      | 0.862     |
| 22        | 0.5735    | 8.36      | 0.5428    |
| 23        | 0.6143    | 7.444     | 1.0       |
| 24        | 0.6143    | 6.346     | 1.0       |
| 25        | 0.6143    | 5.738     | 1.0       |
| 26        | 0.5618    | 5.958     | 0.4959    |
| 27        | 0.6143    | 5.109     | 1.0       |
| 28        | 0.5508    | 5.179     | 0.4566    |
| 29        | 0.6143    | 4.624     | 1.0       |
| 30        | 0.5452    | 1.138     | 0.2272    |
| 31        | 0.5372    | 3.22      | 0.3518    |
| 32        | 0.6062    | 2.713     | 1.0       |
| 33        | 0.6143    | 3.665     | 1.0       |
| 34        | 0.6137    | 7.438     | 0.9997    |
| 35        | -0.02395  | 9.014     | 0.0001    |


| 15        | 0.6395    | 1.781     | 0.8782    |
| 16        | 0.6105    | 1.074     | 0.4212    |
| 17        | 0.6187    | 9.583     | 0.5332    |
| 18        | 0.5857    | 6.95      | 0.3156    |
| 19        | 0.6508    | 6.896     | 0.8346    |
| 20        | 0.535     | 0.2811    | 0.7502    |
| 21        | 0.6281    | 1.023     | 1.0       |
| 22        | 0.5926    | 8.379     | 0.351     |
| 23        | 0.6489    | 7.43      | 1.0       |
| 24        | 0.6489    | 4.858     | 1.0       |
| 25        | 0.6489    | 6.14      | 1.0       |
| 26        | 0.6189    | 5.51      | 0.5416    |
| 27        | -0.03884  | 3.146     | 0.0001    |
| 28        | 0.6489    | 3.489     | 1.0       |
| 29        | 0.6489    | 2.824     | 1.0       |
| 30        | 0.5682    | 1.138     | 0.2272    |
| 31        | 0.5827    | 3.22      | 0.3518    |
| 32        | 0.6287    | 7.693     | 0.5814    |
| 33        | 0.6334    | 6.398     | 0.6105    |
| 34        | 0.6489    | 3.481     | 1.0       |
